In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from src.data_processing import load_data, basic_info

# Wczytanie danych
df = load_data('../data/SMSSpamCollection')

# Podstawowe informacje
basic_info(df)

FileNotFoundError: Plik nie został znaleziony: data/SMSSpamCollection

In [1]:
import pandas as pd
print(pd.__version__)

2.2.3
